In [3]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
tfd = tfp.distributions

# DGP
To begin, we will be generating data consistent with ordinary least squares:

$$ 𝐲= x\beta + \epsilon $$
where 
- 𝐲 is an 𝑁×1 vector of dependent variables, 
- 𝐱 is an 𝑁×𝐾 vector of dependent variables, 
- $\beta$ are the 𝐾×1 parameters we are trying to estimate, 
- $\epsilon$ is an 𝑁×1 vector of iid mean zero homoskedastic unobservables having variance $\sigma^2$.

Here we generate data for 𝑁=500
 and 𝐾=2:

In [5]:
# set tensorflow data type
dtype = tf.float32

##
## simple OLS Data Generation Process
##
# True beta
b = np.array([10, -1])
N = 500
# True error std deviation
sigma_e = 1

x = np.c_[np.ones(N), np.random.randn(N)]
y = x.dot(b) + sigma_e * np.random.randn(N)

Run OLS with the DGP to be compared with MLE

In [6]:
# estimate parameter vector, errors, sd of errors, and se of parameters
bols = np.linalg.inv(x.T.dot(x)).dot(x.T.dot(y))
err = y - x.dot(bols)
sigma_ols = np.sqrt(err.dot(err)/(x.shape[0] - x.shape[1]))
se = np.sqrt(err.dot(err)/(x.shape[0] - x.shape[1]) * np.diagonal(np.linalg.inv(x.T.dot(x))))
# put results together for easy viewing
ols_parms = np.r_[bols, sigma_ols]
ols_se = np.r_[se, np.nan]
print(pd.DataFrame(np.c_[ols_parms, ols_se],columns=['estimate', 'std err'],
      index=['b0', 'b1', 'sigma']))

        estimate   std err
b0     10.044457  0.044706
b1     -0.940323  0.045264
sigma   0.997629       NaN


Getting Data into TF tensors

In [7]:
X = tf.constant(x, dtype=dtype)
Y = tf.constant(y, dtype=dtype)
print(type(X))
print(X.dtype)
print(X.shape)
# extract the first 5 tensors into numpy array
print(X.numpy()[:5])

<class 'tensorflow.python.framework.ops.EagerTensor'>
<dtype: 'float32'>
(500, 2)
[[ 1.         -0.8671768 ]
 [ 1.         -0.335465  ]
 [ 1.         -0.32457274]
 [ 1.         -2.2033095 ]
 [ 1.          0.37562507]]


# Log-Likelihood in TF
Next let's define the log-likelihood function using tensorflow primitives. It goes without saying that we could use tensorflow_probability.distributions.normal for this problem.

$$ LogL =\sum_{i=1}^N log (p(y_i|x_i,b,s)) = \sum_{i=1}^N log (\frac{1}{\sqrt{2\pi s^2}}e^{\frac{-(y_i-x_ib)^2}{2s^2}})$$

$$ LogL =-\frac{N}{2}log(2\pi s^2)-\sum_{i=1}^N\frac{1}{2s^2(y_i-x_ib)^2} $$

In [8]:
pi = tf.constant(np.pi, dtype=dtype)

@tf.function
def ols_loglike(beta, sigma):
    """
    This function defines the negative log likelihood function.
    Argu:
        beta (list): the list of floats as the coefficients of X
        sigma (float): float number as the standard deviation of residuals
    Return:
        ll (EagerTensor): calculated negative log likelihood
    """
    # xb (mu_i for each observation)
    mu = tf.linalg.matvec(X, beta)
    # this is normal pdf logged and summed over all observations
    ll = - (X.shape[0]/2.)*tf.math.log(2.*pi*sigma**2) -\
	    (1./(2.*sigma**2.))*tf.math.reduce_sum((Y-mu)**2., axis=-1)
    return ll

In [9]:
ols_loglike(beta = [1., 1.], sigma=1.)

2023-12-22 08:45:36.570711: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2023-12-22 08:45:36.582962: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


<tf.Tensor: shape=(), dtype=float32, numpy=-22626.936>

## Auto Differenciation
Auto differentiation is possible since tensorflow in the background has defined the model graph given data for finding $\frac{\partial{LogL}}{\partial{\beta}}$ and $\frac{\partial{LogL}}{\partial{\alpha}}$. Evaluating the function and derivative at $\beta =[1., 1.]$
 and $\sigma=1$

In [10]:
[funval, grads] = tfp.math.value_and_gradient(ols_loglike, [[1., 1.], 1.])
print("Function Value: ", funval)
print('Gradients on beta: \n', grads[0])
print('Gradient on sigma: \n', grads[1])

Function Value:  tf.Tensor(-22626.936, shape=(), dtype=float32)
Gradients on beta: 
 [<tf.Tensor: shape=(), dtype=float32, numpy=4583.1455>, <tf.Tensor: shape=(), dtype=float32, numpy=-1230.3246>]
Gradient on sigma: 
 tf.Tensor(43834.934, shape=(), dtype=float32)


# Maximum Likelihood Estimation in Tensorflow
For finding the maximum likelihood estimate, we'll be minimizing the negative log-likelihood since tensorflow has an optimization library based on minimization (rather than maximizing the log-likelihood). Since we will be using a minimizer, we need to construct a negative log-likelihood function in tensorflow. Additionally, we need all of the parameters (the $\beta$'s and $\sigma$) to enter as a single vector.

In [11]:
# how to calculate gradiants
# objective function for minimization
@tf.function
def neg_ols_loglike(param_vec):
   """
   The function that uses gradiants descent to minimize the negative log likelihood function.
   Argu:
      param_vec: vector of parameters as the initial value of [beta,sigma]
   Returns:
      Maximized Log Likelihood.
   """
   beta_split, sigma_split = tf.split(param_vec, [2,1], axis=0)
   # need to take these back down to vectors and scalars:
   beta_split = tf.reshape(beta_split,(2,))
   sigma_split = tf.reshape(sigma_split,())
   # xb (mu_i for each observation)
   mu = tf.linalg.matvec(X, beta_split)
   # this is normal pdf logged and summed over all observations
   ll =  -(X.shape[0]/2.)*tf.math.log(2.*pi*sigma_split**2.) -\
	   (1./(2.*sigma_split**2.))*tf.math.reduce_sum((Y-mu)**2., axis=-1)
   return -1*ll

# return function value and gradients
@tf.function
def neg_like_and_gradient(parms):
    return tfp.math.value_and_gradient(neg_ols_loglike, parms)

In [12]:
# using the same values as above
testval = tf.constant([1., 1., 1.], shape=(3))
out = neg_like_and_gradient(parms=testval)
print("Function value: ", out[0])
print("Gradients: ", out[1])

Function value:  tf.Tensor(22626.936, shape=(), dtype=float32)
Gradients:  tf.Tensor([ -4583.1455   1230.3246 -43834.934 ], shape=(3,), dtype=float32)


An important note that may possibly save someone alot of time: for the optimizer code below to work, the function values and gradients returned by neg_like_and_gradient must be the correct shape.

In [13]:
# set some naiive starting values
start = [0., 0., 1.]

# optimization
optim_results = tfp.optimizer.bfgs_minimize(
    neg_like_and_gradient, start, tolerance=1e-8)

# organize results
est_params = optim_results.position.numpy()
est_serr = np.sqrt(np.diagonal(optim_results.inverse_hessian_estimate.numpy()))
print(pd.DataFrame(np.c_[est_params, est_serr],columns=['estimate', 'std err'],
      index=['b0', 'b1', 'sigma']))

        estimate   std err
b0     10.044456  0.043704
b1     -0.940323  0.045031
sigma   0.995632  0.031042


We can confirm the maximum likelihood solution by inspecting the returned gradient information in optim_results, or by calculating directly

In [14]:
out = neg_like_and_gradient(est_params)
print("Function value: ", out[0])
print("Gradients: ", out[1])

Function value:  tf.Tensor(707.2802, shape=(), dtype=float32)
Gradients:  tf.Tensor([-2.0015240e-04  8.4400177e-05  6.1035156e-05], shape=(3,), dtype=float32)
